In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

dlp_jan_2025_nppe_3_path = kagglehub.competition_download('dlp-jan-2025-nppe-3')

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from torchvision import transforms as T
from tqdm import tqdm
from math import log10

In [ ]:
# ======================
# Dataset Class
# ======================
class SRDataset(Dataset):
    def __init__(self, low_dir, high_dir, transform=None):
        self.low_paths = sorted([os.path.join(low_dir, x) for x in os.listdir(low_dir) if x.endswith('.png')])
        self.high_paths = sorted([os.path.join(high_dir, x) for x in os.listdir(high_dir) if x.endswith('.png')])
        self.transform = transform

    def __len__(self):
        return len(self.low_paths)

    def __getitem__(self, idx):
        low_img = Image.open(self.low_paths[idx]).convert('RGB')
        high_img = Image.open(self.high_paths[idx]).convert('RGB')

        if self.transform:
            low_img = self.transform(low_img)
            high_img = self.transform(high_img)

        return low_img, high_img

transform = T.Compose([
    T.ToTensor()
])

In [ ]:
import torch
import torch.nn as nn

# Residual block with Batch Normalization for better noise handling
class ResidualBlock(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.block = nn.Sequential(
            nn.Conv2d(dim, dim, 3, 1, 1),
            nn.BatchNorm2d(dim),
            nn.ReLU(inplace=True),
            nn.Conv2d(dim, dim, 3, 1, 1),
            nn.BatchNorm2d(dim)
        )
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        return self.relu(x + self.block(x))

# Simple channel attention module to recalibrate feature responses
class ChannelAttention(nn.Module):
    def __init__(self, in_channels, reduction=16):
        super().__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Sequential(
            nn.Linear(in_channels, in_channels // reduction, bias=False),
            nn.ReLU(inplace=True),
            nn.Linear(in_channels // reduction, in_channels, bias=False),
            nn.Sigmoid()
        )

    def forward(self, x):
        b, c, _, _ = x.size()
        y = self.avg_pool(x).view(b, c)
        y = self.fc(y).view(b, c, 1, 1)
        return x * y

# Progressive upsampling block (2x upsampling per block)
class UpSampleBlock2x(nn.Module):
    def __init__(self, in_channels):
        super().__init__()
        self.conv = nn.Conv2d(in_channels, in_channels * 4, 3, 1, 1)
        self.pixel_shuffle = nn.PixelShuffle(2)
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        x = self.conv(x)
        x = self.pixel_shuffle(x)
        return self.relu(x)

# Enhanced model that integrates denoising and 4x super-resolution.
class EnhancedSwinIR(nn.Module):
    def __init__(self, in_channels=3, embed_dim=64, num_blocks=8):
        super().__init__()
        self.head = nn.Conv2d(in_channels, embed_dim, 3, 1, 1)
        self.body = nn.Sequential(*[ResidualBlock(embed_dim) for _ in range(num_blocks)])
        self.attention = ChannelAttention(embed_dim)
        # Progressive upsampling: two consecutive 2x upsamplings achieve 4x magnification.
        self.upsample = nn.Sequential(
            UpSampleBlock2x(embed_dim),
            UpSampleBlock2x(embed_dim)
        )
        self.tail = nn.Conv2d(embed_dim, in_channels, 3, 1, 1)

    def forward(self, x):
        x_head = self.head(x)
        x_body = self.body(x_head)
        x_body = self.attention(x_body)
        # Adding a global residual connection reinforces low-frequency details.
        x_body = x_body + x_head
        x_up = self.upsample(x_body)
        return self.tail(x_up)


In [ ]:
# ======================
# PSNR Metric
# ======================
def calculate_psnr(sr, hr):
    mse = F.mse_loss(sr, hr)
    return 20 * torch.log10(1.0 / torch.sqrt(mse))

# ======================
# Training Loop
# ======================
def train_model(model, train_loader, val_loader, device, epochs=10):
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
    criterion = nn.MSELoss()

    best_psnr = 0
    for epoch in range(epochs):
        model.train()
        train_loss = 0
        for low, high in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}"):
            low, high = low.to(device), high.to(device)
            output = model(low)
            loss = criterion(output, high)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            train_loss += loss.item()

        print(f"\n🔧 Train Loss: {train_loss / len(train_loader):.4f}")

        # Validation PSNR
        model.eval()
        val_psnr = 0
        with torch.no_grad():
            for low, high in val_loader:
                low, high = low.to(device), high.to(device)
                output = model(low)
                val_psnr += calculate_psnr(output, high).item()

        avg_psnr = val_psnr / len(val_loader)
        print(f"📈 Validation PSNR: {avg_psnr:.2f} dB")

        if avg_psnr > best_psnr:
            best_psnr = avg_psnr
            torch.save(model.state_dict(), 'best_model.pth')
            print("✅ Model saved!")

In [ ]:
# ======================
# Run Everything
# ======================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_dataset = SRDataset("/kaggle/input/dlp-jan-2025-nppe-3/archive/train/train", "/kaggle/input/dlp-jan-2025-nppe-3/archive/train/gt", transform=transform)
val_dataset = SRDataset("/kaggle/input/dlp-jan-2025-nppe-3/archive/val/val", "/kaggle/input/dlp-jan-2025-nppe-3/archive/val/gt", transform=transform)

train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=4)

model = EnhancedSwinIR().to(device)
train_model(model, train_loader, val_loader, device, epochs=20)


Epoch 1/20: 100%|██████████| 277/277 [02:54<00:00,  1.59it/s]



🔧 Train Loss: 0.0021
📈 Validation PSNR: 34.19 dB
✅ Model saved!


Epoch 2/20: 100%|██████████| 277/277 [02:32<00:00,  1.82it/s]



🔧 Train Loss: 0.0007
📈 Validation PSNR: 34.89 dB
✅ Model saved!


Epoch 3/20: 100%|██████████| 277/277 [02:32<00:00,  1.82it/s]


...


🔧 Train Loss: 0.0003
📈 Validation PSNR: 37.56 dB


Epoch 19/20: 100%|██████████| 277/277 [02:31<00:00,  1.83it/s]



🔧 Train Loss: 0.0003
📈 Validation PSNR: 37.95 dB


Epoch 20/20: 100%|██████████| 277/277 [02:32<00:00,  1.82it/s]



🔧 Train Loss: 0.0003
📈 Validation PSNR: 37.75 dB


In [ ]:
import torch
from PIL import Image
import os
from torchvision import transforms as T
from tqdm import tqdm

# Load model
model = SwinIRLike().to(device)
model.load_state_dict(torch.load('best_model.pth', map_location=device))
model.eval()

# Image preprocessing
transform = T.Compose([T.ToTensor()])
inv_transform = T.ToPILImage()

# Create output folder
os.makedirs("predicted", exist_ok=True)

# Predict and save
test_dir = "/kaggle/input/dlp-jan-2025-nppe-3/archive/test"
save_dir = "predicted"

with torch.no_grad():
    for fname in tqdm(sorted(os.listdir(test_dir))):
        if fname.endswith('.png'):
            img_path = os.path.join(test_dir, fname)
            image = Image.open(img_path).convert('RGB')
            input_tensor = transform(image).unsqueeze(0).to(device)

            output = model(input_tensor).clamp(0, 1)
            output_image = inv_transform(output.squeeze().cpu())
            output_image.save(os.path.join(save_dir, fname))


NameError: name 'SwinIRLike' is not defined

In [ ]:
import shutil
import os

# Remove old path_to_images folder if exists
if os.path.exists("path_to_images"):
    shutil.rmtree("path_to_images")

# Copy predicted folder to path_to_images
shutil.copytree("predicted", "path_to_images")


In [ ]:
import runpy

runpy.run_path('/kaggle/input/dlp-jan-2025-nppe-3/submission.py')
